### Imports and function definitions

In [1]:
import tensorflow as tf
print('tensorflow version: ', tf.__version__)

import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
import tensorflow_datasets as tfds
print('tfds version: ', tfds.__version__)
import pandas as pd
from tensorflow.keras.callbacks import TensorBoard
import datetime as datetime
from IPython import display
import seaborn as sns

tensorflow version:  2.4.1
tfds version:  4.2.0


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  2
GPUs:  2


### Ways to iterate over the dataset

### Load and Prepare the Dataset / Preprocessing Functions

In [3]:
#https://www.tensorflow.org/datasets/keras_example

(ds_train_raw, ds_test_raw), ds_info = tfds.load(
    'celeb_a',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,)

def preprocess(data):
    """Normalizes images: `uint8` -> `float32`."""
    image = data['image']
    label = data['attributes']
    NUM_BOXES = 1
    boxes = [[.1,.1,.9,.9,]]
    box_indices = tf.zeros(shape=(NUM_BOXES,), dtype=tf.int32)
    #image = tf.image.resize(image[20:-20], [64, 64], antialias=True)
    image = tf.image.crop_and_resize(tf.expand_dims(image, axis=0), crop_size=[64, 64], boxes=boxes, box_indices=box_indices)
    image = tf.cast(image, tf.float32) / 255. # normalise images to between 0 and 1
    image = image[0]
    return image, label

ds_train = ds_train_raw.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)

ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [4]:
# for data in ds_train:
#     for i in range(10):
#         plt.imshow(data[0][i])
#         plt.show()
#     break

In [5]:
# Get attributes list
for images, attributes in ds_train:
    break
    
# Checks
print(images.shape)
print(images[0].shape)
print(attributes.keys())
print(images.numpy().max())
print(images.numpy().min())

(128, 64, 64, 3)
(64, 64, 3)
dict_keys(['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young'])
1.0
0.0


## Create the models

Adapated from https://www.tensorflow.org/tutorials/generative/dcgan which uses MNIST to create a non-conditional GAN

### Generator

The generator uses tf.keras.layers.Conv2DTranspose (upsampling) layers to produce an image from a seed (random noise). Start with a Dense layer that takes this seed as input, then upsample several times until you reach the desired image size of 64x64x1. Notice the tf.keras.layers.LeakyReLU activation for each layer, except the output layer which uses tanh.

In [6]:
# The generator
def make_generator_model():
    
    #input_attributes = layers.Input(shape=(40,))
    #input_noise = tf.random.normal([128, 24])
    #merge = layers.Concatenate()([input_attributes, input_noise])
    
    model = tf.keras.Sequential(name='Generator')
    # add noise layer
    #model.add(layers.GaussianNoise(stddev=0.1, input_shape=(40,)))
    
    # Densly connection NN layer of 8*8*256 (16,384) neurons. This is taking in the input and multiplying with the weights
    model.add(layers.Dense(8*8*256, use_bias=False)) 
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # 8 x 8 image with 256 features.
    model.add(layers.Reshape((8, 8, 256)))
    #assert model.output_shape == (None, 8, 8, 256)  # Note: None is the batch size

    # Compress channels/features to 128
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False)) # channels=128, filter size=5*5
    #assert model.output_shape == (None, 8, 8, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Upsampling block
    model.add(layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', use_bias=False))
    #assert model.output_shape == (None, 16, 16, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # Upsampling block
    model.add(layers.Conv2DTranspose(32, (4, 4), strides=(2, 2), padding='same', use_bias=False))
    #assert model.output_shape == (None, 32, 32, 32)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(16, (4, 4), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.LeakyReLU())
    
    # Output layer - Activation of sigmoid is used to squash values between 0 and 1 in line with image values after preprocessing
    model.add(layers.Conv2D(3, (3,3), activation='sigmoid', padding = 'same', use_bias=False))
    #assert model.output_shape == (None, 64, 64, 3)
    
    #output = model(merge)
    
    #model = tf.keras.Model(inputs=[input_attributes], outputs=output, name='Generator_2')

    return model

### Discriminator

The discriminator is a CNN-based image classifier.

In [7]:
def make_discriminator_model():
    
    # label input
    attribute_vector = layers.Input(shape=(40+24))
    input_image = layers.Input(shape=(64,64,3))
    
    conv_model = tf.keras.Sequential(name='Discriminator_conv_model')
    
    # gan hacks - add noise to help generate a variety of images
    conv_model.add(layers.GaussianNoise(stddev=0.1))
    
    # downsample - increase filters as it progresses through the model
    conv_model.add(layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same'))
    conv_model.add(layers.BatchNormalization())
    conv_model.add(layers.LeakyReLU())
    #assert conv_model.output_shape == (None, 32, 32, 64)
    
    #downsample
    conv_model.add(layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same'))
    conv_model.add(layers.BatchNormalization())
    conv_model.add(layers.LeakyReLU())
    
    #downsample
    conv_model.add(layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same'))
    conv_model.add(layers.LeakyReLU())
    conv_model.add(layers.Dropout(0.3))
    
    # classifier - flatten to 1D array
    conv_model.add(layers.Flatten())
    
    # Push image through model above, shape=[batch,dimensions(128*8*8)]
    img_feature_vec = conv_model(input_image)
    
    # concatenate layers - the image vector + the attribute labels, shape=[batch, dimensions(128*8*8+40)]
    merge = layers.Concatenate()([img_feature_vec, attribute_vector])
    
    # Gan Hacks - deeper model
    dense_model = tf.keras.Sequential(name='Discriminator_deep')
    dense_model.add(layers.Dense(128))
    #dense_model.add(layers.Dense(256))
    dense_model.add(layers.LeakyReLU())
    dense_model.add(layers.Dropout(0.3))
    dense_model.add(layers.Dense(1))

    # Push image + attributes through model and make a prediction
    output = dense_model(merge)
    
    # Create new model here with correct shapes which need to be defined as inputs
    model = tf.keras.Model(inputs=[input_image, attribute_vector], outputs=output, name='Discriminator')

    return model

In [8]:
# Create Models
# Classify the generated images as real or fake using the discriminator.
# Model is trained to output positive values for real images and negative values for fake images
generator = make_generator_model()
discriminator = make_discriminator_model()

#decision = discriminator((generated_image, attributes_noise))
#print(decision.shape)
#print(decision[:10])

## Define the loss and optimisers

In [9]:
# This method returns a helper function to compute cross entropy loss
# The discriminator has binary input (fake or real data) so a binary loss function is used
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

### Discriminator loss

Quantifies how well the discriminator is able to distinguish real images from fakes.  

It compares the discriminator's predictions on real images to an array of 1s, and the discriminator's predictions on fake (generated) images to an array of 0s

In [10]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

### Generator loss

Quantifies how well it was able to trick the discriminator.  

If the generator is performng well, the discriminator will classify the fake images as real (or 1). This function compares the discriminators decisions on the generated images to an array of 1s

In [11]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

### Optimisers

Two different optimisers required since the two networks are trained separately

In [12]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

### Save checkpoints

In [13]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Training Loop

The training loop begins with generator receiving a random seed as input. That seed is used to produce an image. The discriminator is then used to classify real images (drawn from the training set) and fakes images (produced by the generator). The loss is calculated for each of these models, and the gradients are used to update the generator and discriminator.

In [14]:
def convert_attrbutes_to_vec_tf(attributes):
    # Convert to tensor - transpose as tensorflow looks at batch first
    batch_attributes_as_bool_tensor = tf.transpose(tf.stack(list(attributes.values())))
    # Convert from boolean to float - true = 1, false = -1
    batch_attributes_as_float_tensor = tf.where(batch_attributes_as_bool_tensor, 1.,-1.)
    input_noise = tf.random.normal([128, 24])
    attributes_input = tf.concat([batch_attributes_as_float_tensor, input_noise], axis=-1)
    return attributes_input

In [15]:
@tf.function #execute code in graph mode for efficiency
def train_step(images, attributes):

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        
        # Generate noise to pass to generator
        attribute_input = convert_attrbutes_to_vec_tf(attributes)

        # Make generated image and pass to discriminator for prediction
        generated_images = generator(attribute_input, training=True)
        real_output = discriminator((images, attribute_input), training=True)
        fake_output = discriminator((generated_images, attribute_input), training=True)

        # Calculate loss
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
        
        # Keep an eye on what discriminator is deciding (visualised in tensorboard)
        mean_real_output = tf.math.reduce_mean(tf.nn.sigmoid(real_output))
        mean_fake_output = tf.math.reduce_mean(tf.nn.sigmoid(fake_output))

    # Calculate gradients - the derivative of the loss with respect to the model weights
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    # Tweak model weights based on optimiser calculations
    # zip function is used to align the array of 128 (or diff batch size) gradients with the 128 trainable variables that the gradients will update
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return attribute_input, gen_loss, disc_loss, mean_real_output, mean_fake_output
    
def train(dataset, epochs, num_examples_to_generate):
    
    # tensorboard callback
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    log_dir='tensorboard_log/' + current_time
    summary_writer = tf.summary.create_file_writer(log_dir)
        
    for epoch in range(epochs):
        start = time.time()

        for image_batch, attribute_batch in dataset:
            attribute_input, gen_loss, disc_loss, mean_real_output, mean_fake_output = train_step(image_batch, attribute_batch)

        # Produce images for the GIF as you go
        display.clear_output(wait=True)
        generate_and_save_images(generator, epoch + 1, attribute_input[:num_examples_to_generate])
        
        # save data for tensorboard
        with summary_writer.as_default():
            tf.summary.scalar('gen_loss', gen_loss, step=epoch)
            tf.summary.scalar('disc_loss', disc_loss, step=epoch)
            tf.summary.scalar('mean_real_output', mean_real_output, step=epoch)
            tf.summary.scalar('mean_fake_output', mean_fake_output, step=epoch)

        # Save the model every 50 epochs
        if (epoch + 1) % 50 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
    
# Generate and save images
def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False. This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(10, 10))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i])
        plt.axis('off')

    image_folder = 'generated_images/'
    image_filepath = image_folder + 'image_at_epoch_{:04d}.png'
    plt.savefig(image_filepath.format(epoch))
    plt.show()


## Train the model

Generator and discriminator are trained simultaneously.  

It is important that the generator and discriminator do not overpower each other (i.e. they train at a similar rate)

At the beginning of the training, the generated images look like random noise. As training progresses, the generated digits will look increasingly real. After about 50 epochs, they resemble MNIST digits. This may take about one minute / epoch with the default settings on Colab.

In [ ]:
restore_dir = checkpoint_dir + '/20210508-162827'
#checkpoint.restore(tf.train.latest_checkpoint(restore_dir))

EPOCHS = 250
noise_dim = 100
num_examples_to_generate = 16

train(ds_train, EPOCHS, num_examples_to_generate)

In [ ]:
assert False

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

# Attribute Synonym Mapping

In order to process text to generate an image, we need to search the text for attributes. 

The Celeb_A dataset has 40 attributes which we have mapped to additional synonyms.

We have then created a function to take the text and search for attributes by first looking for trigrams and iterating down to unigrams

In [ ]:
# Dictionary of synonyms aligned with attributes per Celeb_A dataset

dict_attribute_to_synonym = {
    'Attractive': ['attractive', 'beautiful', 'alluring', 'glamorous', 'lovely', 'inviting', 'fair', 'enticing', 
                   'interesting', 'charming', 'pleasant', 'good-looking', 'tempting', 'gorgeous', 'pretty',
                   'engaging', 'pleasing', 'handsome', 'adorable', 'agreeable', 'comely', 'enchanting'],
    'Bags_Under_Eyes': ['bags under eyes', 'dark circles under eyes'],
    'Bald': ['bald', 'hairless', 'baldheaded', 'no hair', 'little hair', 'abrupt', 'shaven', 'depilated', 'plain', 
             'direct', 'stark', 'bald headed', 'glabrous', 'austere', 'downright', 'forthright', 'outright'],
    'Big_Lips': ['big lips', 'pouty lips', 'round lips', 'plump lips', 'puffy lips', 'full lips', 
                 'fat lips', 'big mouth', 'massive mouth', 'thick lips', 'fleshy lips', 'puffy lips', 'big-lipped'],
    'Bangs': ['bangs', 'fringe'],
    'Big_Nose': ['big nose', 'big-nosed', 'large nose', 'schnoz', 'huge nose'],
    'Black_Hair': ['black hair', 'jet black hair', 'inky hair', 'pitch-black hair', 'coal-black hair', 'inky-black hair', 'jet-black hair', 
                   'raven hair'],
    'Blond_Hair': ['blond hair', 'fair hair', 'blonde hair', 'yellow hair', 'golden hair', 'light hair', 'sandy hair', 
                   'bottle blond hair', 'honey blond hair', 'golden-yellow hair', 'platinum blond hair', 
                   'platinum hair', 'sandy blond hair'],
    'Brown_Hair': ['brown hair', 'chestnut hair', 'dark brown hair', 'brownish hair', 'light brown hair', 
                   'caramel brown hair', 'dark hair', 'walnut brown hair', 'brown-haired hair', 
                   'chocolate brown hair', 'chocolate hair', 'chocolate coloured hair', 'cinnamon brown hair', 
                   'hazel hair', 'maple brown hair'],
    'Bushy_Eyebrows': ['bushy eyebrows', 'thick eyebrows', 'hairy eyebrows', 'heavy eyebrows', 'intense eyebrows', 
                       'massive eyebrows'],
    'Chubby':['chubby', 'plump', 'tubby', 'pudgy', 'rotund', 'overweight', 'chunky', 'portly', 'stout', 'fleshy', 'corpulent', 
              'fat', 'dumpy', 'obese', 'beefy', 'heavy', 'meaty', 'gross', 'bulky', 'fat man', 'full-figured', 'hefty', 
              'thick'],
    'Double_Chin': ['double chin', 'double-chinned', 'double chinned'],
    'Eyeglasses': ['eyeglasses', 'glasses', 'spectacles'],
    'Goatee': ['goatee', 'anchor beard', 'balbo beard', 'facial hair'],
    'Gray_Hair': ['gray hair', 'grey hair', 'grey haired', 'white hair', 'salt and pepper hair', 'salt-and-pepper', 
                  'grizzled', 'white haired', 'angling', 'gray haired', 'gray-haired', 'silver hair', 'greying hair', 
                  'white-haired'],
    'Heavy_Makeup': ['heavy makeup', 'thick makeup', 'full makeup'],
    'High_Cheekbones': ['high cheekbones', 'prominent cheekbones'],
    'Male': ['male', 'man', 'he', 'guy','gentleman', 'dude'],
    'Mouth_Slightly_Open': ['open mouth'],
    'Mustache': ['mustache', 'whisker', 'mustachio'],
    'Narrow_Eyes': ['narrow eyes', 'little eyes', 'slender eyes', 'small eyes', 'blinkered eyes', 'skinny eyes'],
    'Oval_Face': ['oval face', 'spherical face', 'oval smile', 'round look', 'round face'],
    'No_Beard': ['no beard', 'beardless', 'clean shaven', 'shaved'],
    'Pale_Skin': ['pale skin', 'white skin', 'pale', 'pallor', 'blonde skin', 'fair skin', 'ghost like', 'ghostly skin', 'ghostly complextion'],
    'Pointy_Nose': ['pointy nose', 'pointed nose', 'pointy root', 'sharp nose', 'keen nose'],
    'Receding_Hairline': ['receding hairline', 'getting bald', 'going bald', 'little hair', 'alopecia', 'losing hair', 'balding'],
    'Rosy_Cheeks': ['rosy cheeks', 'red cheek', 'flushed face', 'pink face', 'bright cheek'],
    'Sideburns': ['sideburns', 'shiskers', 'side-whiskers', 'mutton chop'],
    'Smiling': ['smiling', 'smile', 'grin', 'happy', 'grinning', 'cheerful', 'smiled'],
    'Straight_Hair':['straight hair', 'sleek hair', 'straight hair style'],
    'Wavy_Hair': ['wavy hair', 'curly hair', 'curls', 'curled hair'],
    'Earrings': ['earrings', 'earing', 'studs'],
    'Hat': ['hat', 'bonnet', 'cap', 'helmet', 'hood', 'beret'],
    'Lipstick': ['lipstick', 'lip rouge', 'lip gloss', 'makeup', 'chapstick'],
    'Necklace': ['necklace', 'pendant', 'choker', 'necklet', 'bead', 'pearls'],
    'Necktie': ['necktie', 'tie', 'cravat', 'bow tie', 'black tie'],
    'Young': ['young', 'youth', 'youthful', 'junior', 'youger', 'children']
}

# Create reversed dictionary of synonym to attribute
dict_synonym_to_attribute = {}
for key, value in dict_attribute_to_synonym.items():
    for sub_value in value:
        dict_synonym_to_attribute[sub_value] = key

In [ ]:
#dict_synonym_to_attribute
dict_synonym_to_attribute['pretty']

In [ ]:
# Function to process text to attributes

def get_attributes_from_text(text):
    """
    process text to extract attributes matched to celeb_a dataset
    input: string
    output: list of attributes as strings
    """
    text = text.split(' ')
    attr_list = []
    
    # Check for triigrams and delete from description if present
    for i in reversed(range(len(text) - 2)):

        try:
            attr_list.append(dict_synonym_to_attribute[' '.join([text[i], text[i+1], text[i+2]])])
            del text[i]
            del text[i]
            del text[i]
        except:
            pass
    
    # Check for bigrams and delete from description if present
    for i in reversed(range(len(text) - 1)):

        try:
            attr_list.append(dict_synonym_to_attribute[' '.join([text[i], text[i+1]])])
            del text[i]
            del text[i]
        except:
            pass

    # Check for unigrams and delete from description if present
    for i in reversed(range(len(text))):
        try:
            attr_list.append(dict_synonym_to_attribute[text[i]])
            del text[i]
        except:
            pass
        
    return attr_list

In [ ]:
test = 'the young man with black hair entered the room'
test = 'the pretty girl with blond hair smiled at me'
test = 'the pretty girl with chocolate coloured hair smiled at me'
get_attributes_from_text(test)

# Generate images according to attributes

In [ ]:
attributes.keys()

In [ ]:
# Create dictionary of attributes to bool value
def create_attribute_base_dict():
    attribute_base_dict = {}

    for idx, attribute in enumerate(attributes.keys()):
        attribute_base_dict[attribute] = False
        
    return attribute_base_dict

attribute_base_dict = create_attribute_base_dict()

In [ ]:
def attributes_bool_ordered(attribute_base_dict, attribute_to_find):
    """
    order boolean values in same order as received by model
    output: list of boolean values
    """
    attribute_to_find = list(attribute_to_find)
    
    attribute_values = []
    for attribute in sorted(attribute_base_dict.keys()):
        attribute_values.append(True if attribute in attribute_to_find else False)
        
    return attribute_values

def attributes_multiple_list(attribute_to_find):
    """
    create list of lists of attributes in ordered bool form
    output: list of lists of boolean values
    """
    if any(isinstance(item, list) for item in attribute_to_find):
        attribute_values = []
        for attribute_list in attribute_to_find:
            attribute_values.append(attributes_bool_ordered(attribute_base_dict, attribute_list))
        
    else:
        attribute_values = attributes_bool_ordered(attribute_base_dict, attribute_to_find)
    return attribute_values

def convert_attrbutes_to_vec(attributes):
    """
    prepare attributes for image generation
    input: list of ordered boolean values
    output: tensor of shape (num_images, 40)
    """
    # Order attributes
    #attributes = attributes_bool_ordered(attribute_base_dict, attributes)
    attributes = attributes_multiple_list(attributes)
    print('attributes length', len(attributes))
    
    # Convert to tensor
    #print('p1: ',attribute_embeddings.shape, type(attribute_embeddings))
    batch_attributes_as_bool_tensor = tf.stack(list(attributes))
    print('p2: ',batch_attributes_as_bool_tensor.shape, type(batch_attributes_as_bool_tensor))
    if len(batch_attributes_as_bool_tensor.shape) == 1:
        num_images = 1
    else:
        num_images = batch_attributes_as_bool_tensor.shape[0]
    batch_attributes_as_bool_tensor = tf.reshape(batch_attributes_as_bool_tensor, [num_images,40])
    print('p2: ',batch_attributes_as_bool_tensor.shape, type(batch_attributes_as_bool_tensor))
    
    # Convert from boolean to float
    batch_attributes_as_float_tensor = tf.where(batch_attributes_as_bool_tensor, 1.,-1.)
    print('p3', batch_attributes_as_float_tensor.shape, type(batch_attributes_as_float_tensor))
    
    # Create projection of attributes into embedding space
    #attribute_input = batch_attributes_as_float_tensor
    
    input_noise = tf.random.normal([num_images, 24])
    #input_noise = tf.zeros([num_images, 24])
    attributes_input = tf.concat([batch_attributes_as_float_tensor, input_noise], axis=-1)
    return attributes_input

In [ ]:
#attribute_to_test = ['Eyeglasses', 'Male', 'Attractive', 'Blond_Hair']
attribute_to_test = [['Eyeglasses', 'Male', 'Attractive'], ['Eyeglasses', 'Male', 'Attractive', 'Blond_Hair'], ['Eyeglasses', 'Male', 'Blond_Hair'], 
                     ['Attractive', 'Blond_Hair'],['Attractive', 'Blond_Hair', 'Wearing_Lipstick']]

attribute_to_test = [['Eyeglasses', 'Male', 'Attractive'], ['Eyeglasses', 'Male', 'Attractive'], ['Eyeglasses', 'Male', 'Attractive'],
                     ['Eyeglasses', 'Male', 'Attractive'], ['Eyeglasses', 'Male', 'Attractive']]

#attribute_to_test = [['Eyeglasses', 'Attractive'], ['Eyeglasses', 'Attractive', 'Blond_Hair'], ['Eyeglasses', 'Blond_Hair'], 
 #                    ['Attractive', 'Blond_Hair'],['Attractive', 'Blond_Hair', 'Wearing_Lipstick']]

#attribute_to_test = [['Attractive'], ['Attractive', 'Blond_Hair'], ['Blond_Hair'], 
 #                    ['Attractive', 'Brown_Hair'],['Attractive', 'Blond_Hair', 'Wearing_Lipstick']]


attributes_test = convert_attrbutes_to_vec(attribute_to_test)
print(attributes_test.shape, type(attributes_test))

In [ ]:
# Wrapper function for text to attributes
def convert_text_to_vector(text):
    attributes_list = get_attributes_from_text(text)
    attributes = convert_attrbutes_to_vec(attributes_list)
    return attributes

In [ ]:
# Generating an image based on specific attributes

attributes_test = convert_attrbutes_to_vec(attribute_to_test)
generated_image = generator(attributes_test, training=False)

fig = plt.figure(figsize=(10, 10))

num_images = generated_image.shape[0]

for i in range(num_images):
    cols = 5
    rows = int(np.ceil(num_images / cols))
    plt.subplot(rows, cols, i+1)
    plt.imshow(generated_image[i])
    plt.axis('off')

None

In [ ]:
test = 'the young man with black hair entered the room'
test = 'the pretty girl with blond hair smiled at me'
#test = 'the pretty girl with chocolate coloured hair smiled at me'
test = 'the bald girl smiled at me'
test = convert_text_to_vector(test)

In [ ]:
# Generating an image based on text

generated_image = generator(test, training=False)

fig = plt.figure(figsize=(10, 10))

num_images = generated_image.shape[0]

for i in range(num_images):
    cols = 5
    rows = int(np.ceil(num_images / cols))
    plt.subplot(rows, cols, i+1)
    plt.imshow(generated_image[i])
    plt.axis('off')

None

# Distribution of Attributes

In [ ]:
attributes_distribution = []

for images, attributes in ds_train:
    attributes_distribution.append(attributes)

In [ ]:
df = pd.DataFrame(columns = attributes.keys())

for batch in attributes_distribution:
    df_temp = pd.DataFrame(batch)
    df = df.append(df_temp)
    
print(len(df))

#df = df * 1

In [ ]:
df_count = pd.DataFrame(df.sum()).reset_index()
df_count.columns = ['Attribute', 'Count']
#df_count

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))
sns.barplot(x='Attribute', y='Count', data=df_count)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
None

In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
    return PIL.Image.open('./generated_images/image_at_epoch_{:04d}.png'.format(epoch_no))

display_image(50)

In [ ]:
attributes_random = np.random.choice(a=[False, True], size=(128,40))
print(attributes_random[1].shape)
attributes_random[1]

In [ ]:
generator.summary()

In [ ]:
discriminator.summary()

# References

- Dataset - https://www.tensorflow.org/datasets/catalog/celeb_a
- Model design - https://www.tensorflow.org/tutorials/generative/dcgan
- Model design - https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-a-cifar-10-small-object-photographs-from-scratch/
- Model design - Conditional Gan - https://machinelearningmastery.com/how-to-develop-a-conditional-generative-adversarial-network-from-scratch/
- Model design - https://machinelearningmastery.com/how-to-train-a-progressive-growing-gan-in-keras-for-synthesizing-faces/
- Gan Hacks - Improving Gan Performance - https://machinelearningmastery.com/how-to-code-generative-adversarial-network-hacks/
- Adding noise - https://machinelearningmastery.com/how-to-improve-deep-learning-model-robustness-by-adding-noise/